## Modelado de datos para Funciones de Machine Learning

### Importando librerías 

In [1]:
import pandas as pd
import numpy as np

import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator

import fastparquet as fp 
import pyarrow as pa
import pyarrow.parquet as pq


### Modelo de recomendación Item_Item

In [2]:
#Carga del archivo
modelo_item= pd.read_parquet("Data/modelo_item.parquet")
modelo_item.head(3)

,item_id,app_name,genres
88310,761140,Lost Summoner Kitty,Action
88310,761140,Lost Summoner Kitty,Casual
88310,761140,Lost Summoner Kitty,Indie


In [3]:
#Creacíon de dummies
modelo_item= pd.get_dummies(modelo_item, columns=["genres"], prefix="")

modelo_item= modelo_item.groupby(["item_id","app_name"]).sum().reset_index()

modelo_item.head(4)

,item_id,app_name,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_RPG,_Racing,_Simulation,_Sin Datos,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Calcular la similitud del coseno

Se calcula la similitud del coseno entre todos los pares posibles de ítems en función de sus perfiles. La similitud del coseno mide el ángulo entre dos vectores en un espacio multidimensional y varía de -1 (completamente opuestos) a 1 (idénticos). Cuanto mayor sea la similitud del coseno entre dos ítems, más parecidos se considerarán.

In [4]:
similitudes = cosine_similarity(modelo_item.iloc[:,3:])

In [5]:
similitudes.shape

(32131, 32131)

In [6]:
#Almaceno el df para la funcion:
modelo_item.to_parquet("Data/modelo_item.parquet")

In [7]:
def recommend_games(id: int):
    '''
    Esta función recomienda 5 juegos a partir del juego ingresado.

    Args:
        game_id (int): ID único del videojuego al cual se le harán las recomendaciones.
    '''
    game = modelo_item[modelo_item['item_id'] == id]

    if game.empty:
        return(f"El juego '{id}' no posee registros.")
    
    # Obtiene el índice del juego dado
    idx = game.index[0]


    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_item.iloc[idx, 3:]], modelo_item.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = modelo_item['app_name'].iloc[similar_game_indices].tolist()

    return {f"Juegos similares a {modelo_item[modelo_item['item_id'] == id]['app_name'].values[0]}: {', '.join(similar_game_names)}"}

In [8]:
recommend_games(10)

{'Juegos similares a Counter-Strike: Team Fortress Classic, Day of Defeat, Deathmatch Classic, Half-Life: Opposing Force, Ricochet'}